<a href="https://colab.research.google.com/github/vivekchadaram/NLP-word-predictor-model/blob/main/NLP_word_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import tensorflow as tf
import string
import requests
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
def dataclean(data):
  datatokens=data.split()
  condition=str.maketrans('','',string.punctuation)
  datatokens=[w.translate(condition) for w in datatokens]
  datatokens = [ w for w in datatokens if w.isalpha()]
  datatokens=[w.lower() for w in datatokens]
  return datatokens

In [3]:
text=requests.get("https://www.gutenberg.org/cache/epub/5200/pg5200.txt","")
text.text

'\ufeffThe Project Gutenberg EBook of Metamorphosis, by Franz Kafka\r\nTranslated by David Wyllie.\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **\r\n**     Please follow the copyright guidelines in this file.     **\r\n\r\n\r\nTitle: Metamorphosis\r\n\r\nAuthor: Franz Kafka\r\n\r\nTranslator: David Wyllie\r\n\r\nRelease Date: August 16, 2005 [EBook #5200]\r\nFirst posted: May 13, 2002\r\nLast updated: May 20, 2012\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK METAMORPHOSIS ***\r\n\r\n\r\n\r\n\r\nCopyright (C) 2002 David Wyllie.\r\n\r\n\r\n\r\n\r\n\r\n  Metamorphosis\r\n  Franz Kafka\r\n\r\nTranslated by David Wyllie\r\n\r\n\r\n\r\nI\r\n\r\n\r\nOne morning, when Gregor

In [4]:
text=text.text.split("\n")


In [5]:
text[46]

'One morning, when Gregor Samsa woke from troubled dreams, he found\r'

In [6]:
text=text[46:]

In [7]:
text

['One morning, when Gregor Samsa woke from troubled dreams, he found\r',
 'himself transformed in his bed into a horrible vermin.  He lay on\r',
 'his armour-like back, and if he lifted his head a little he could\r',
 'see his brown belly, slightly domed and divided by arches into stiff\r',
 'sections.  The bedding was hardly able to cover it and seemed ready\r',
 'to slide off any moment.  His many legs, pitifully thin compared\r',
 'with the size of the rest of him, waved about helplessly as he\r',
 'looked.\r',
 '\r',
 '"What\'s happened to me?" he thought.  It wasn\'t a dream.  His room,\r',
 'a proper human room although a little too small, lay peacefully\r',
 'between its four familiar walls.  A collection of textile samples\r',
 'lay spread out on the table - Samsa was a travelling salesman - and\r',
 'above it there hung a picture that he had recently cut out of an\r',
 'illustrated magazine and housed in a nice, gilded frame.  It showed\r',
 'a lady fitted out with a fur hat a

In [8]:
data = " ".join(text)

In [9]:
data

'One morning, when Gregor Samsa woke from troubled dreams, he found\r himself transformed in his bed into a horrible vermin.  He lay on\r his armour-like back, and if he lifted his head a little he could\r see his brown belly, slightly domed and divided by arches into stiff\r sections.  The bedding was hardly able to cover it and seemed ready\r to slide off any moment.  His many legs, pitifully thin compared\r with the size of the rest of him, waved about helplessly as he\r looked.\r \r "What\'s happened to me?" he thought.  It wasn\'t a dream.  His room,\r a proper human room although a little too small, lay peacefully\r between its four familiar walls.  A collection of textile samples\r lay spread out on the table - Samsa was a travelling salesman - and\r above it there hung a picture that he had recently cut out of an\r illustrated magazine and housed in a nice, gilded frame.  It showed\r a lady fitted out with a fur hat and fur boa who sat upright,\r raising a heavy fur muff that c

In [10]:
cleaneddata=dataclean(data)

In [11]:
cleaneddata

['one',
 'morning',
 'when',
 'gregor',
 'samsa',
 'woke',
 'from',
 'troubled',
 'dreams',
 'he',
 'found',
 'himself',
 'transformed',
 'in',
 'his',
 'bed',
 'into',
 'a',
 'horrible',
 'vermin',
 'he',
 'lay',
 'on',
 'his',
 'armourlike',
 'back',
 'and',
 'if',
 'he',
 'lifted',
 'his',
 'head',
 'a',
 'little',
 'he',
 'could',
 'see',
 'his',
 'brown',
 'belly',
 'slightly',
 'domed',
 'and',
 'divided',
 'by',
 'arches',
 'into',
 'stiff',
 'sections',
 'the',
 'bedding',
 'was',
 'hardly',
 'able',
 'to',
 'cover',
 'it',
 'and',
 'seemed',
 'ready',
 'to',
 'slide',
 'off',
 'any',
 'moment',
 'his',
 'many',
 'legs',
 'pitifully',
 'thin',
 'compared',
 'with',
 'the',
 'size',
 'of',
 'the',
 'rest',
 'of',
 'him',
 'waved',
 'about',
 'helplessly',
 'as',
 'he',
 'looked',
 'whats',
 'happened',
 'to',
 'me',
 'he',
 'thought',
 'it',
 'wasnt',
 'a',
 'dream',
 'his',
 'room',
 'a',
 'proper',
 'human',
 'room',
 'although',
 'a',
 'little',
 'too',
 'small',
 'lay',
 'pe

In [12]:
seqlen=40+1
lines=[]
for i in range(seqlen,len(cleaneddata)):
  line=" ".join(cleaneddata[i-seqlen:i])
  lines.append(line)

In [13]:
len(lines)

24859

In [14]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
seq=tokenizer.texts_to_sequences(lines)

In [15]:
seq=np.array(seq)

In [16]:
seq

array([[  69,  161,   61, ...,  818, 1071,  391],
       [ 161,   61,   15, ..., 1071,  391, 1529],
       [  61,   15,  104, ...,  391, 1529,    3],
       ...,
       [3009, 3010, 3011, ..., 3020,    2,  269],
       [3010, 3011, 3012, ...,    2,  269,   58],
       [3011, 3012,   27, ...,  269,   58,  268]])

In [17]:
x,y = seq[:,:-1],seq[:,-1]
x,y

(array([[  69,  161,   61, ...,    5,  818, 1071],
        [ 161,   61,   15, ...,  818, 1071,  391],
        [  61,   15,  104, ..., 1071,  391, 1529],
        ...,
        [3009, 3010, 3011, ...,  820, 3020,    2],
        [3010, 3011, 3012, ..., 3020,    2,  269],
        [3011, 3012,   27, ...,    2,  269,   58]]),
 array([ 391, 1529,    3, ...,  269,   58,  268]))

In [18]:
inputlen=x.shape[1]

In [19]:
len(set(cleaneddata)),len(tokenizer.word_index)

(3020, 3020)

In [20]:
vocabulary=len(set(cleaneddata))+1
y= to_categorical(y,vocabulary)

In [21]:
model=Sequential()
model.add(Embedding(vocabulary,40,input_length=inputlen))
model.add(LSTM(500,return_sequences=True))
model.add(LSTM(500))
model.add(Dense(500,activation='relu'))
model.add(Dense(vocabulary,activation='softmax'))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 40)            120840    
                                                                 
 lstm (LSTM)                 (None, 40, 500)           1082000   
                                                                 
 lstm_1 (LSTM)               (None, 500)               2002000   
                                                                 
 dense (Dense)               (None, 500)               250500    
                                                                 
 dense_1 (Dense)             (None, 3021)              1513521   
                                                                 
Total params: 4,968,861
Trainable params: 4,968,861
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [24]:
model.fit(x,y,batch_size=64,epochs=150)

Epoch 1/150
389/389 [==============================] - 32s 61ms/step - loss: 6.3451 - accuracy: 0.0515
Epoch 2/150
389/389 [==============================] - 24s 61ms/step - loss: 5.9838 - accuracy: 0.0567
Epoch 3/150
389/389 [==============================] - 24s 61ms/step - loss: 5.8080 - accuracy: 0.0666
Epoch 4/150
389/389 [==============================] - 24s 61ms/step - loss: 5.6744 - accuracy: 0.0731
Epoch 5/150
389/389 [==============================] - 24s 61ms/step - loss: 5.5457 - accuracy: 0.0826
Epoch 6/150
389/389 [==============================] - 23s 60ms/step - loss: 5.4170 - accuracy: 0.0904
Epoch 7/150
389/389 [==============================] - 23s 60ms/step - loss: 5.2818 - accuracy: 0.1000
Epoch 8/150
389/389 [==============================] - 23s 60ms/step - loss: 5.1500 - accuracy: 0.1105
Epoch 9/150
389/389 [==============================] - 23s 60ms/step - loss: 5.0261 - accuracy: 0.1208
Epoch 10/150
389/389 [==============================] - 23s 60ms/step - l

In [28]:
def generate_text_seq(model,tokenizer,inputlen,seed_text,n):
  text=[]
  for i in range(n):
    encoded=tokenizer.texts_to_sequences([seed_text])[0]
    encoded=pad_sequences([encoded],maxlen=inputlen,truncating='pre')
    encoded=np.array(encoded)
    y_predict=model.predict(encoded)
    m=np.argmax(y_predict,axis=1)
    predicted_word=""
    for word,index in tokenizer.word_index.items():
      if index==m[0]:
        predicted_word=word
        break
    seed_text=seed_text+" "+predicted_word
    text.append(predicted_word)
  return " ".join(text)

In [33]:
seed_text="Oh, God\", he thought, \"what a strenuous career it is that I've\
chosen! Travelling day in and day out.  Doing business like this\
takes much more effort than doing your own business at home, and on\
top of that"
n=int(input("Enter the no. of words you want to predict: "))
generate_text_seq(model,tokenizer,inputlen,lines[1],n)

Enter the no. of words you want to predict: 40


'and divided by arches into stiff sections the bedding was hardly able to cover it and seemed ready to slide off any moment his many legs pitifully thin compared with the size of the rest of him waved about helplessly'